## Delaunay Triangulation and Voronoi Diagrams

- Overview
- What is Delaunay Triangulation?
- What is a Voronoi Diagram ?
- Delaunay Triangulation Code
- Delaunay Triangulation Animation Code

# What is Delaunay Triangulation?

1. Cover entire space
2. No point should be inside the circumcircle of any triangle

![Delaunay Triangulation](data/images/opcv4face-w4-m3-dealunyTriangle-1-small.png)

![Delaunay triangulation favors small angles](data/images/opcv4face-w4-m3-angles.png)

# What is a Voronoi Diagram?

![Voronoi Diagram](data/images/opcv4face-w4-m3-voronoiDiagram-small.png)

> If you connect the points in neighboring Voronoi regions, you get a Delaunay triangulation!



# Subdiv2D : OpenCV implementation of Delaunay Triangulation


## OpenCV Output 

> x10 y10 x22 y22 x30 y30

## We want 

> 10 22 30


## Step 1: Import required module

In [54]:
import cv2
import numpy as np

## Step 2: Find index of closest landmark point

The function `findIndex` finds the index of the closest landmark point to the given vertex from the vector of points.

In [55]:
def findIndex(points, point):
  diff = np.array(points) - np.array(point)

  # Find the distance of point from all points
  diffNorm = np.linalg.norm(diff, 2, 1)

  # Find the index with minimum distance and return it
  return np.argmin(diffNorm)

## Step 3: Write Delauanay Triangles to a file

The function `writeDelaunay` gets the Delaunay triangles using the `getTriangleList` method of the `subdiv` object.

`getTriangleList` returns a array of 6 floating point numbers. The six numbers are the `x` and `y` coordinates of the three vertices of a triangle.

We are interested in storing the output file as a list of indices of the vertices and not the actual vertices. So, we loop through all the triangles returned by `getTriangleList` and find the index of the vertices.

Each row of the output file contains three number that represent a triangle. The numbers are the indices of points in the input file.

In [56]:
# write delaunay triangles to file
def writeDelaunay( subdiv, points, outputFileName ) :

    # Obtain the list of triangles.
    # Each triangle is stored as vector of 6 coordinates
    # (x0, y0, x1, y1, x2, y2)
    triangleList = subdiv.getTriangleList();
    filePointer = open(outputFileName,'w')
    # Will convert triangle representation to three vertices pt1, pt2, pt3
    for t in triangleList :
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])

        # Find the landmark corresponding to each vertex
        landmark1 = findIndex(points,pt1)
        landmark2 = findIndex(points,pt2)
        landmark3 = findIndex(points,pt3)

        filePointer.writelines("{} {} {}\n".format(landmark1, landmark2, landmark3 ))

    filePointer.close()

## Step 4: Read image

We specify the colors for drawing lines and points on the image. Then we read the image.

In [57]:
# Define colors for drawing.
delaunayColor = (255,255,255)
pointsColor = (0, 0, 255)

# Read in the image.
img = cv2.imread("data/images/smiling-man.jpg");

# Step 5: Calculate Delaunay Triangles

Delaunay triangulation and voronoi diagrams are calculated using the class `Subdiv2D`.

`Subdiv2D` is initialized with a rectangle containing all the points. The landmark points are read from the file.

The `insert` method is used to insert points.

`writeDelaunay` function uses `getTriangleList` method to get the delaunay triangles.

In [58]:
# Rectangle to be used with Subdiv2D
size = img.shape
rect = (0, 0, size[1], size[0])

# Create an instance of Subdiv2D
subdiv = cv2.Subdiv2D(rect);

# Create an array of points.
points = []

# Read in the points from a text file
with open("data/images/smiling-man-delaunay.txt") as file :
    for line in file:
        x, y = line.split()
        points.append((int(x), int(y)))

outputFileName = "results/smiling-man-delaunay.tri"

# Insert points into subdiv
for p in points:
    subdiv.insert(p)

# Step 6: Write Delaunay Triangles to file


In [59]:
writeDelaunay(subdiv, points, outputFileName)
print("Writing Delaunay triangles to {}".format(outputFileName))

Writing Delaunay triangles to results/smiling-man-delaunay.tri


# Delaunay Triangulation Animation Code
It is instructive to see an animation of Delaunay triangulation and Voronoi diagram as the points get added. Let’s dive into the code.

## Step 1: Import required modules

In [60]:
#!/usr/bin/python
import cv2
import numpy as np
import random

## Step 2: Check if a given point falls inside a rectangle

This function checks if a given point falls inside a rectangle. This is the python counter-part of the OpenCV `rect.contains()` method provided in C++.

In [61]:
# Check if a point is inside a rectangle
# Rect is an array of (x, y, w, h)
def rectContains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[2] :
        return False
    elif point[1] > rect[3] :
        return False
    return True

## Step 3: Draw a point on image

The function `drawPoint` draws a point on an image using a specified color.

In [62]:
# Draw a point on the image
def drawPoint(img, p, color ) :
    cv2.circle( img, p, 2, color, -1, cv2.LINE_AA, 0 )

## Step 4: Draw Delaunay Triangles

The function `drawDelaunay` draws Delaunay triangles on an image using a specified color.

Points are inserted using an instance of `Subdiv2D` before calling this function. Delaunay triangles are obtained by calling the method `getTriangleList`.

Once we get the triangles, we draw the lines by going over each vertex of the triangle.

In [63]:
# Draw delaunay triangles
def drawDelaunay(img, subdiv, delaunayColor ) :

    # Obtain the list of triangles.
    # Each triangle is stored as vector of 6 coordinates
    # (x0, y0, x1, y1, x2, y2)
    triangleList = subdiv.getTriangleList();
    size = img.shape
    r = (0, 0, size[1], size[0])

    # Will convert triangle representation to three vertices pt1, pt2, pt3
    for t in triangleList :
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])

        # Draw triangles that are completely inside the image
        if rectContains(r, pt1) and rectContains(r, pt2) and rectContains(r, pt3) :
            cv2.line(img, pt1, pt2, delaunayColor, 1, cv2.LINE_AA, 0)
            cv2.line(img, pt2, pt3, delaunayColor, 1, cv2.LINE_AA, 0)
            cv2.line(img, pt3, pt1, delaunayColor, 1, cv2.LINE_AA, 0)

## Step 5: Draw Voronoi Diagrams

The function `drawVoronoi` draws Voronoi diagrams on an image.

Points are inserted using an instance of `Subdiv2D` before calling this function. The Voronoi diagram is represented as a collection of facets (polygons) which can be obtained using the the method `getVoronoiFacetList`.

The facets so obtained are drawn on the image using `fillConvexPoly` and the boundaries of the facets are drawn using polylines.

In [64]:
# Draw voronoi diagram
def drawVoronoi(img, subdiv) :

    # Get facets and centers
    ( facets, centers) = subdiv.getVoronoiFacetList([])

    for i in range(0,len(facets)) :
        ifacetArr = []
        for f in facets[i] :
            ifacetArr.append(f)

        # Extract ith facet
        ifacet = np.array(ifacetArr, np.int)

        # Generate random color
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        # Fill facet with a random color
        cv2.fillConvexPoly(img, ifacet, color, cv2.LINE_AA, 0);

        # Draw facet boundary
        ifacets = np.array([ifacet])
        cv2.polylines(img, ifacets, True, (0, 0, 0), 1, cv2.LINE_AA, 0)

        # Draw centers.
        cv2.circle(img, (centers[i][0], centers[i][1]), 3, (0, 0, 0), -1, cv2.LINE_AA, 0)

## Step 6: Find index of closest landmark point

The function `findIndex` finds the index of the closest landmark point to the given vertex from the vector of points.

In [65]:
def findIndex(points, point):
  diff = np.array(points) - np.array(point)

  # Find the distance of point from all points
  diffNorm = np.linalg.norm(diff, 2, 1)

  # Find the index with minimum distance and return it
  return np.argmin(diffNorm)

## Step 7: Write Delaunay Triangles to file

Save output Delaunay triangles to file. The function `writeDelaunay` takes as input an instance of `Subdiv2D`, the list of input points, and saves it to disk.

In [66]:
# write delaunay triangles to file
def writeDelaunay( subdiv, points, outputFileName ) :

    # Obtain the list of triangles.
    # Each triangle is stored as vector of 6 coordinates
    # (x0, y0, x1, y1, x2, y2)
    triangleList = subdiv.getTriangleList();
    filePointer = open(outputFileName,'w')
    # Will convert triangle representation to three vertices pt1, pt2, pt3
    for t in triangleList :
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])

        # Find the landmark corresponding to each vertex
        landmark1 = findIndex(points,pt1)
        landmark2 = findIndex(points,pt2)
        landmark3 = findIndex(points,pt3)

        filePointer.writelines("{} {} {}\n".format(landmark1, landmark2, landmark3 ))

    filePointer.close()

## Step 8: Create a bounding box

We specify the colors for drawing lines and points on the image. Then we read the image and create a bounding box named `rect`.

In [67]:
# Define colors for drawing.
delaunayColor = (255,255,255)
pointsColor = (0, 0, 255)

# Read in the image.
img = cv2.imread("data/images/smiling-man.jpg");

# Rectangle to be used with Subdiv2D
size = img.shape
rect = (0, 0, size[1], size[0])

## Step 9: Draw Landmark Points

The image bounding box is used to initialize an instance of `Subdiv2D`. The points are subsequently added to this object for calculating and displaying Delaunay triangulation and Voronoi diagrams.

In [68]:
# Create an instance of Subdiv2D
subdiv = cv2.Subdiv2D(rect);

# Create an array of points.
points = [];

# Allocate space for voronoi Diagram
imgVoronoi = np.zeros(img.shape, dtype = img.dtype)

# Read in the points from a text file
with open("data/images/smiling-man-delaunay.txt") as file :
    for line in file :
        x, y = line.split()
        points.append((int(x), int(y)))

outputFileName = "results/smiling-man-delaunay.tri"

# Draw landmark points on the image
for p in points :
    drawPoint(img, p, pointsColor )

## Step 10: Draw Delaunay Triangulation and Voronoi Diagram

We calculate and draw the Delaunay triangulation ( on `imgDelaunay` ) and Voronoi Diagram on ( `imgVoronoi` ) every time a new point is added thus creating an animation.

In [69]:
# Insert points into subdiv
plotPoints = []
count = 0
for p in points :
    count += 1
    writeFileName = "delaunayAnimation/delaunayAnimation" + str(count).zfill(4) + ".png"
    
    subdiv.insert(p)
    plotPoints.append(p)

    imgDelaunay = img.copy()

    # Draw delaunay triangles and voronoi diagrams
    drawDelaunay(imgDelaunay, subdiv, delaunayColor);
    drawVoronoi(imgVoronoi,subdiv)

    for pp in plotPoints :
        drawPoint(imgDelaunay, pp, pointsColor)

    # Create a horizontal stack
    imgDisplay = np.hstack([imgDelaunay, imgVoronoi])
    # Write image to file
    cv2.imwrite(writeFileName,imgDisplay)

writeDelaunay(subdiv, points, outputFileName)
print("Writing Delaunay triangles to {}".format(outputFileName))

Writing Delaunay triangles to results/smiling-man-delaunay.tri


## Step 11: Create animation from images

To create animation from saved images, we will use ImageMagick's `convert` command.

In [70]:
!convert delaunayAnimation/delaunayAnimation* -loop 0 -delay 100 results/delaunayAnimation.gif
!rm delaunayAnimation/delaunayAnimation*

convert: unable to open image 'delaunayAnimation/delaunayAnimation*': No such file or directory @ error/blob.c/OpenBlob/3457.
convert: no decode delegate for this image format `' @ error/constitute.c/ReadImage/512.
convert: no images defined `results/delaunayAnimation.gif' @ error/convert.c/ConvertImageCommand/3275.
rm: delaunayAnimation/delaunayAnimation*: No such file or directory


# Animation

![Delaunay Animation](results/delaunayAnimation.gif)